In [1]:
from urllib.request import urlopen
import json
import sqlite3
import time

In [14]:
con = sqlite3.connect("hasar_tespit_25_02_2023.db")
cur = con.cursor()

In [15]:
res = cur.execute("""SELECT il.ilid, iladi, ilce.ilceid, ilce.ilceadi, mahalleid, mahalleadi 
FROM il, ilce, mahalle 
WHERE il.ilid = ilce.ilid and ilce.ilceid = mahalle.ilceid""")

In [16]:
il_ilce_mahalle_tbl = res.fetchall()


In [18]:
print(il_ilce_mahalle_tbl[0])

(1, 'Adana', 1757, 'Aladağ', 4232, 'Mansurlu')


In [20]:
for item in il_ilce_mahalle_tbl:
    print(item[5])
    break

Mansurlu


In [ ]:
get_ilce_url = "https://hasartespit.csb.gov.tr/query/counties?cityId=" 
get_mahalle_url = "https://hasartespit.csb.gov.tr/query/districts?countyId="

In [ ]:
verification_token="__RequestVerificationToken=CfDJ8PZLaRZRODdKtgtNmfRIkdeYAjudQhLFJTR4yZ3C81ObwfWbh-9pERcIq2b7yVMpwPy7FVfZbVJD-STac14gRsIo1VC_0IzwTEA64qO2WYwNvjB9FkKy0kFyyUNdv9W5q3YDmK1Izd8NldUzR9l-TLo"
#verification token needs to assigned manually. Open the front page and get the token by selecting il, ilce, mahalle.

ilid_part = "IlId="
ilceid_part = "&IlceId="
mahalleid_part = "&MahalleId="
sokak_bina_askikodu_part="&Sokak=&BinaNo=&AskiKodu=&" + verification_token

hasar_sorgu_url = "https://hasartespit.csb.gov.tr/query/AddressQuery?"


In [ ]:
for item in il_ilce_mahalle_tbl:
    ilid = item[0]
    iladi = item[1]
    ilceid = item[2]
    ilceadi = item[3]
    mahalleid = item[4]
    mahalleadi = item[5]
    print(ilid, iladi, ilceid, ilceadi, mahalleid, mahalleadi)
    sorgu_url = hasar_sorgu_url + ilid_part + str(ilid) + ilceid_part + str(ilceid) + mahalleid_part + str(mahalleid) + sokak_bina_askikodu_part
    response = urlopen(sorgu_url)
    data = json.loads(response.read())
    for hasar_items in data:
        uid = hasar_items["uid"]
        askiKodu = hasar_items["askiKodu"]
        yapi_kimlik_no=hasar_items["yapi_kimlik_no"]
        sokak=hasar_items["sokak"]
        binaNo=hasar_items["binaNo"]
        aciklama=hasar_items["aciklama"]
        itirazSonucu=hasar_items["itirazSonucu"]
        deprem_adi=hasar_items["deprem_adi"]
        deprem_siddeti=hasar_items["deprem_siddeti"]
        deprem_tarihi=hasar_items["deprem_tarihi"]
        fotograflar=str(hasar_items["fotograflar"])
        try:
            cur.execute("INSERT INTO HASAR VALUES(?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", [mahalleid, uid,askiKodu,yapi_kimlik_no,sokak,binaNo,aciklama,itirazSonucu,deprem_adi,deprem_siddeti,deprem_tarihi,fotograflar])
            con.commit()
        except:
            print("Not inserted: ", ilid, iladi, ilceid, ilceadi, mahalleid, mahalleadi)
        time.sleep(1)
    

In [7]:
con.close()